# Manhattan Plot Tutorial

In [ ]:
import caplot
from pprint import pprint
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

## Dataset
The dataset used in this notebook is described in [SampleData.md](https://github.com/ArashLab/caplot/tree/main/examples/data/SampleData.md)

Briefly the `variants.tsv.gz` contains the following columns
- locus-contig: CHR in VCF
- locus-position: POS in VCF
- alleles: List of allele in a JSON string
- REF: first allele in the alleles
- ALT: second allele in the alleles
- rsid
- qual: From 1000 Genome VCF
- filters: From 1000 Genome VCF
- vep-: All the annotations produced by VEP (joined based on rsid)
- variant_qc-: quality-control metrics computed by hail.variant_qc
- maf: Minor Allele Frequency
- LogReg: 3 logistic regression tests are performed on t2d phenotype
    - 1: lrt test with no covariate
    - 2: score test with age and isFemale as covariate
    - 3: wald test with age, isFemale and all 10 pcaMAF vectors as covariate
- LinReg: 3 linear regression tests are performed on bmi phenotype
    - 1: with no covariate
    - 2: with age and isFemale as covariate
    - 3: with age, isFemale and all 10 pcaMAF vectors as covariate

### Create the caplot PCA object

In [ ]:
plot = caplot.Manhattan()

### Load data
You may load data from pandas dataframe, tabular file and SQL database.\
Read the documentaion for this property to see details of suported formats.\
In case, the data source is a file, caplot infer file format from the extension (i.e. `tsv.gz`)

In [ ]:
plot.source = 'data/variants.tsv.gz'

### Access internal data 
caplot store data internally in a pandas dataframe.\
You can access that datafame using `_data`.\
Let see the columns available in the data.

In [ ]:
pprint(list(plot._data.columns))

### Set the requiered attributes
Which columns contains principal components to be ploted?
Basically the X and Y coordinate for a scatter plot

In [ ]:
plot.contig = 'locus-contig'
plot.position = 'locus-position'
plot.pvalue = 'LogReg3-p_value'

### Show the plot

In [ ]:
plot.Show()

### Smaller points
For crowded regions

In [ ]:
plot.pointSize = 2
plot.Show()

### Too many variants?
Let focus on the top 5000 variants.

In [ ]:
plot.pointSize = 4
plot.top = 5000
plot.Show()

### Fit it for your purpose
Wider with more colors and more dots

In [ ]:
plot.coloringPalette = 'Category20'
plot.numColors = 8
plot.height = 300
plot.width = 1200
plot.top = 50000
plot.Show()

In [ ]:
# Revert the settings
plot.coloringPalette = 'Category10'
plot.numColors = 2
plot.height = 600
plot.width = 800
plot.top = 5000

### Filter Variants
Filter variants using SQL queries.\
Here we focus on protein coding regions

In [ ]:
plot.filter = 'SELECT * FROM data WHERE "vep-BIOTYPE"=="protein_coding"'
plot.Show()

### Highligh Variants
Highligh variants using SQL queries.\
In this example we highlight variants where minor allele frequencies are above 10%.\

In [ ]:
plot.highlight = 'SELECT * FROM data WHERE "maf">0.1'
plot.Show()

### More contrast in highlight

In [ ]:
plot.minorAlpha = 0.05
plot.Show()

### Try grey highlight too clearly see both highlighted node and the background

In [ ]:
plot.minorAlpha = 0.3
plot.greyHighlight = True
plot.Show()

### Add Hovers
See variant details instantly when you spot the variant of your interst

In [ ]:
plot.hovers = {'chr': 'locus-contig', 'pos': 'locus-position',
               'gene': 'vep-Gene', 'lof': 'vep-LoFtool',
               'symbol': 'vep-SYMBOL', 'rsid': 'rsid'}
plot.Show()

### Hovers is a python dictionary
Use dictionary functions to modify hovers.

In [ ]:
plot.hovers.pop('gene', None)
plot.hovers['symbol'] = 'vep-SYMBOL'
plot.hovers.update({'lof': 'vep-LoFtool'})
plot.Show()

### Even more interactivity with forms

In [ ]:
plot.ShowWithForm()

### Ultimate interactivity with customized form
In this example `c1` to `c4` are value selectors.\
`c1` and `c2` are used in the filter query.\
`c3` and `c4` are used in the highlight query.

`c1` selects the super population.\
`c2` selects the MAF (Minor Allele Frequency).\
`c3` selects the BMI.\
`c4` selects the Gender.

Also we color the samples by the sub-population


In [ ]:
c1 = '{Minimum MAF:floatSlider:0:0.5:0.01:0.1}'
c2 = '{Impact to Highligh:singleChoice:"vep-IMPACT":"MODERATE"}'
c3 = '{BioType to Highligh:singleChoice:"vep-BIOTYPE":"protein_coding"}'

plot.filterTemplate = f'SELECT * FROM data WHERE "maf" > {c1}'
plot.highlightTemplate = f'SELECT * FROM data WHERE "vep-IMPACT" = {c2} OR "vep-BIOTYPE" = {c3} '

plot.minorAlpha = 0.1
plot.greyHighlight = True

plot.ShowWithForm()

### Save your plot in a variety of formats
The format is infered from the file extension.\
Hovers and bokeh tooltip remains active in the html output.\
Note that when you save as HTML it activate output_file().\
As a cnsequence all your subsequent plot.Show() will show the plot in the notebook as well as in a new tab (file).\
To prevent this behaviour you need to reset output and then activate output notebook.

In [ ]:
plot.SaveAs('results/manhattan.png')
plot.SaveAs('results/manhattan.jpeg')
plot.SaveAs('results/manhattan.svg')
plot.SaveAs('results/manhattan.pdf')

In [ ]:
plot.SaveAs('results/manhattan.html')

In [ ]:
output_notebook()

### Save plot, data and config all together.
It is possible to save everything in one file and share it.\
Use `caplot` as your file extension. That's all. 

In [ ]:
plot.SaveAs('results/manhattan.caplot')

### Restore everything.
To test this feature you can restart your notebook (clear all data) and run the following cell.\
It will restore your plot, data and config all together.

In [ ]:
plot = caplot.read('results/manhattan.caplot')

In [ ]:
plot.ShowWithForm()

### Directly play with bokeh plot
Change the title by accessing underying bokeh object

In [ ]:
# revert to single plot
plot.subplots = ['pcaMAF-scores_1', 'pcaMAF-scores_2']
plot.subplotWidth = 400
plot.subplotHeight = 400

bokeh_plot = plot.Generate()
bokeh_plot.title = 'Manhattan Plot'
show(bokeh_plot)

### Do it all at once
You can set allmost all parameters in the init function

In [ ]:
plot = caplot.Manhattan(source = 'data/variants.tsv.gz',
                        contig = 'locus-contig',
                        position = 'locus-position',
                        pvalue = 'LogReg1-p_value',
                        filter = 'SELECT * FROM data WHERE "maf">0.01',
                        highlight = 'SELECT * FROM data WHERE "maf">0.1',
                        top = 10000,
                        yRange = (0,6))
plot.Show()

### Get Vep Annotation on the Fly

In [ ]:
plot.VEPLimit=10
plot.rsidColumn = 'rsid'

In [ ]:
plot._annotationData